# Spanner Workbench Sneak Peak

## installation

In [ ]:
%%bash
# git clone https://github.com/DeanLight/spanner_workbench
# cd spanner_workbench
# pip install -e src/rgxlog-interpreter 

## Datalog magic

In [1]:
%load_ext autoreload
%autoreload 2

import rgxlog
from rgxlog import magic_client,Session
import rgxlog.magic.rgxlog_magic

In [2]:
%%spanner
# define relation and some facts about it
new parent(str ,str)
parent("bob", "greg")
parent("greg", "alice")
parent("greg", "janice")
# now add a rule that deduces that bob is a grandparent of alice
grandparent(X,Z) <- parent(X,Y), parent(Y,Z) # ',' is a short hand to the 'and' operator

In [3]:
%%spanner
# now for the queries
?parent("greg", ChildsOfGreg) # returns all children of greg
?grandparent(X, Y) # returns all tuples in the grandparent relation

printing results for query 'parent("greg", ChildsOfGreg)':
  ChildsOfGreg
----------------
     janice
     alice

printing results for query 'grandparent(X, Y)':
  X  |   Y
-----+--------
 bob | janice
 bob | alice



## python IO

In [4]:
! cat sons.csv

bob,adam
adam,oren
jane,bob
elmo,bernie
bernie,casie

In [5]:
magic_client.import_relation_from_csv("sons.csv",relation_name="parent",delimiter=",")

In [6]:
%%spanner
?parent(X,Y) # returns all tuples that

printing results for query 'parent(X, Y)':
   X    |   Y
--------+--------
 bernie | casie
  elmo  | bernie
  jane  |  bob
  adam  |  oren
  bob   |  adam
  greg  | janice
  greg  | alice
  bob   |  greg



In [7]:
%%spanner
?grandparent(X, Y)

printing results for query 'grandparent(X, Y)':
  X   |   Y
------+--------
 elmo | casie
 jane |  adam
 jane |  greg
 bob  |  oren
 bob  | janice
 bob  | alice



In [8]:
grand_parents=magic_client.query_into_df("?grandparent(Grandparent, Grandchild)")
grand_parents

,Grandparent,Grandchild
0,elmo,casie
1,jane,adam
2,jane,greg
3,bob,oren
4,bob,janice
5,bob,alice


## Adding ie functions - regexes

In [9]:
%load_ext autoreload
%autoreload 2

from rgxlog.engine.datatypes.primitive_types import DataTypes
from rgxlog.stdlib.regex import rgx_string,rgx_string_out_types



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
magic_client.register(
                ie_function=rgx_string,
                ie_function_name='RGXString',
                in_rel=[DataTypes.string, DataTypes.string],
                out_rel=rgx_string_out_types,
                )

In [11]:
%%spanner
bank_records= "bob 100 elmo 80 jane 79 alice 60"
money(P, Money) <- \
RGXString(bank_records, "(\w+).*?(\d+)")->(P, Money)



In [12]:
%%spanner
?money(P,M)

printing results for query 'money(P, M)':
   P   |   M
-------+-----
 alice |  60
 jane  |  79
 elmo  |  80
  bob  | 100



## NLP example

In [13]:
%load_ext autoreload
%autoreload 2

from rgxlog.stdlib.nlp import   lemma_wrapper, ner_wrapper, dependency_parse_wrapper                                


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
sentence = "John fathered Greg wonderfully"
print("NER")
print(list(ner_wrapper(sentence)))
print("lemma")
print(list(lemma_wrapper(sentence)))
print("dep_parse")
print(list(dependency_parse_wrapper(sentence)))


NER
[('John', 'PERSON', (0, 4)), ('Greg', 'PERSON', (14, 18))]
lemma
[('John', 'John', (0, 4)), ('fathered', 'father', (5, 13)), ('Greg', 'Greg', (14, 18)), ('wonderfully', 'wonderfully', (19, 30))]
dep_parse
[('ROOT', 0, 'ROOT', 2, 'fathered'), ('nsubj', 2, 'fathered', 1, 'John'), ('obj', 2, 'fathered', 3, 'Greg'), ('advmod', 2, 'fathered', 4, 'wonderfully')]


In [15]:
magic_client.register(ie_function=lemma_wrapper,
             ie_function_name='Lemma',
             in_rel=[DataTypes.string],
             out_rel=[DataTypes.string, DataTypes.string, DataTypes.span],
             )

magic_client.register(ie_function=ner_wrapper,
           ie_function_name='NER',
           in_rel=[DataTypes.string],
           out_rel=[DataTypes.string, DataTypes.string, DataTypes.span],
           )



magic_client.register(ie_function=dependency_parse_wrapper,
                ie_function_name='DepParse',
                in_rel=[DataTypes.string],
                out_rel=[DataTypes.string, DataTypes.integer, DataTypes.string, DataTypes.integer, DataTypes.string]
                )




In [16]:
%%spanner
sentence="John fathered Greg wonderfully"

#TODO multiline rule doesnt parse

father_of(F,S)<- NER(sentence)->(F,"PERSON",T1),NER(sentence)->(S,"PERSON",T2),Lemma(sentence)->(FATHERLIKE ,"father",T3),DepParse(sentence)->("nsubj",T41,FATHERLIKE,T42,F),DepParse(sentence)->("obj",T51,FATHERLIKE,T52,S)
#NER(sentence)->(F,"PERSON",T1)
#NER(sentence)->(S,"PERSON",T2)
#Lemma(sentence)->(FATHERLIKE ,"father",T3)
#DepParse(sentence)->("nsubj",T41,FATHERLIKE,T42,F)
#DepParse(sentence)->("obj",T51,FATHERLIKE,T52,S)


In [18]:
%%spanner

?father_of(F,S)

printing results for query 'father_of(F, S)':
  F   |  S
------+------
 John | Greg

